# Лабораторная работа №3

# Фундаментальные концепции искусственного интеллекта

Студент **Морозов Артем Борисович**

Группа **М8О-109СВ-24**

3. Оптимизация гиперпараметров 
     - С помощью [optuna]() взять пример, аналогичный третьему туториалу документации, используя sklearn и с другим датасетом, выбрать другие  алгоритмы классификации 
        и кластеризации не из туториала  и визуализировать графики для полученного процесса
        - В качестве других моделей подойдут любые алгоритмы классификации и регрессии из sklearn которые не использовались в туториале
     - Использовать 2 разных семплера и прунера
     - При процессе оптимизации гиперпараметров использовать общую память через postgreSQL
     - В качестве отчёта выступают: исходный код, инструкция запуска реляционной БД. 

In [15]:
import optuna
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

In [4]:
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
study_classification = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study_clustering = optuna.create_study(direction="maximize", sampler=optuna.samplers.RandomSampler(), pruner=optuna.pruners.MedianPruner())

[I 2024-12-22 23:52:43,996] A new study created in memory with name: no-name-463fd641-5775-4d5a-b914-7dd041494323
[I 2024-12-22 23:52:43,998] A new study created in memory with name: no-name-7a38fb35-39ef-43aa-a2fc-d67d42e6f465


In [18]:
def objective_classification(trial):
    classifier_name = trial.suggest_categorical("classifier", ["SVC", "RandomForest"])

    if classifier_name == "SVC":
        svc_c = trial.suggest_loguniform("svc_c", 1e-3, 1e3)
        kernel = trial.suggest_categorical("kernel", ["linear", "rbf"])
        classifier = SVC(C=svc_c, kernel=kernel, random_state=42)

    elif classifier_name == "RandomForest":
        n_estimators = trial.suggest_int("n_estimators", 10, 100)
        max_depth = trial.suggest_int("max_depth", 2, 20)
        classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)

    score = cross_val_score(classifier, X_train, y_train, cv=3, scoring="accuracy").mean()
    return score

In [19]:
def objective_clustering(trial):
    clustering_name = trial.suggest_categorical("clustering", ["KMeans", "DBSCAN"])

    if clustering_name == "KMeans":
        n_clusters = trial.suggest_int("n_clusters", 2, 10)
        model = KMeans(n_clusters=n_clusters, random_state=42)
    elif clustering_name == "DBSCAN":
        eps = trial.suggest_loguniform("eps", 1e-3, 1.0)
        min_samples = trial.suggest_int("min_samples", 2, 10)
        model = DBSCAN(eps=eps, min_samples=min_samples)

    labels = model.fit_predict(X)
    score = silhouette_score(X, labels) if len(set(labels)) > 1 else -1
    return score

In [ ]:
study_classification.optimize(objective_classification, n_trials=50)
study_clustering.optimize(objective_clustering, n_trials=50)

C:\Users\artem\AppData\Local\Temp\ipykernel_8264\760220596.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  svc_c = trial.suggest_loguniform("svc_c", 1e-3, 1e3)
[I 2024-12-22 23:53:06,712] Trial 0 finished with value: 0.9499999999999998 and parameters: {'classifier': 'SVC', 'svc_c': 0.02080195393307515, 'kernel': 'linear'}. Best is trial 0 with value: 0.9499999999999998.
[I 2024-12-22 23:53:07,328] Trial 1 finished with value: 0.9583333333333334 and parameters: {'classifier': 'RandomForest', 'n_estimators': 71, 'max_depth': 12}. Best is trial 1 with value: 0.9583333333333334.
[I 2024-12-22 23:53:07,670] Trial 2 finished with value: 0.9583333333333334 and parameters: {'classifier': 'RandomForest', 'n_estimators': 53, 'max_depth': 20}. Best is trial 1 with value: 0.9583333333333334.
C:\Users\artem\AppData\Local\Temp\ipykernel_

In [21]:
print("Best hyperparameters for classification:")
print(study_classification.best_params)

print("Best hyperparameters for clustering:")
print(study_clustering.best_params)

Best hyperparameters for classification:
{'classifier': 'SVC', 'svc_c': 0.9890210625560284, 'kernel': 'linear'}
Best hyperparameters for clustering:
{'clustering': 'DBSCAN', 'eps': 0.9530910029763359, 'min_samples': 2}


In [ ]:
optuna.visualization.plot_optimization_history(study_classification).show()
optuna.visualization.plot_optimization_history(study_clustering).show()